## PUMS Microdata Personas Basic Processing
This notebook consumes Public Use Microdata Sample files (PUMS) to create an aggregated and normalized file that includes household and person level data that can be used for analysis and answer specific questions about different types of users of the transportation system.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# PUMS IDs for each selected PUMA (Seattle: Downtown, Northeast, Northwest, Southeast, and West)
SEATTLE_PUMS = [11601, 11602, 11603, 11604, 11605]

# Loading household data for Seattle-only locations
df_household = pd.read_csv('ss16hwa.csv')
df_household = df_household[df_household['PUMA'].isin(SEATTLE_PUMS)]

# Loading person data for Seattle-only locations
df_person = pd.read_csv('ss16pwa.csv')
df_person = df_person[df_person['PUMA'].isin(SEATTLE_PUMS)]

### Processing Person level data

In [3]:
# Filter the data set for important indices to track 
df_person = df_person[['PUMA', 'SERIALNO', 'RAC1P', 'AGEP', 'DDRS', 'DEAR', 'DEYE',
                        'DOUT', 'DPHY', 'ENG', 'PINCP', 'HISP','PWGTP',
                      'JWMNP','JWTR', 'MIG', 'SCHL', 'SEX']]

In [4]:
ps = df_person[['PUMA', 'SERIALNO', 'AGEP']].copy()

In [5]:
# Categorizing the PUMA sections & creating dummy variables
ps['PUMA'] = ps['PUMA'].replace([11601, 11602, 11603, 11604, 11605], ['NORTHWEST', 'NORTHEAST',
                                                                                'DOWNTOWN', 'SOUTHEAST', 'WEST'])
# Crating a dummy variable for PUMA 
dummy_puma = pd.get_dummies(ps['PUMA'])
dummy_puma.loc[ps['PUMA'].isnull(), :] = np.nan
ps = pd.concat([ps, dummy_puma], axis = 1)

# Categorizing RAC1P (preparing for clustering)
"""
RAC1P-RACE: 
    0: White alone
    1: Black or African American alone
    2: American Indian Alone & Alaska Native alone
    3: Asian alone 
    4: Native Hawaiian & other pacific islander
    5: Some other race alone & Two or more races 
"""
ps['RAC1P'] = df_person['RAC1P'].replace([1, 2, 3, 4, 5, 6, 7, 8, 9], ['WHITE_ALONE', 'BLACK_ALONE', 
                                                                       'NATIVE_INDIAN', 'NATIVE_INDIAN', 
                                                                       'NATIVE_INDIAN', 'ASIAN_ALONE', 
                                                                       'PACIFIC_ISLANDER', 'OTHER_RACE', 'OTHER_RACE'])
# Crating a dummy variable for RACE
dummy_mode = pd.get_dummies(ps['RAC1P'])
dummy_mode.loc[ps['RAC1P'].isnull(), :] = np.nan
ps = pd.concat([ps, dummy_mode], axis = 1)

#  Categorizing SEX (preparing for clustering)
"""
SEX: 
    0: Male
    1: Female
"""
ps['SEX'] = df_person['SEX'].replace([1, 2], ['MALE', 'FEMALE'])

# Crating a dummy variable for SEX (male vs. female)
dummy_mode = pd.get_dummies(ps['SEX'])
dummy_mode.loc[ps['SEX'].isnull(), :] = np.nan
ps = pd.concat([ps, dummy_mode], axis = 1)

# Categorizing travel time to work(preparing for clusterign) --> TRAVEL_TIME_TO_WORK = 'JWMNP'
ps['JWMNP'] = pd.to_numeric(df_person['JWMNP'], errors='coerce').fillna(0).astype(np.int64)

In [6]:
# Categorizing the hispanic (hispanic origin of any race) sections (preparing for clustering)
""" 
HISP: 
    0: Not hispanic origin
    1: Hispanic origin
"""
ps['HISP'] = df_person['HISP'].replace([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17,
                                        18, 19, 20, 21, 22, 23, 24], [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 
                                                        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

# Categorizing the DDRS (Self-care difficulty) sections (preparing for clustering)
"""
DDRS-SELF_CARE_DIFFICULTY: 
    0: Yes
    1: No 
"""
ps['DDRS'] = df_person['DDRS'].replace([1, 2], [0, 1])

# Categorizing the DEAR (hearing difficulty) sections (preparing for clustering)
"""
DEAR-HEARING_DIFFICULTY: 
    0: Yes
    1: No
"""
ps['DEAR'] = df_person['DEAR'].replace([1, 2], [0, 1])

# Categorizing the DEYE (vision difficulty) sections (preparing for clustering)
"""
DEYE-VISION_DIFFICULTY:
    0: Yes
    1: No
"""
ps['DEYE'] = df_person['DEYE'].replace([1, 2], [0, 1])

# Categorizing the DOUT (independent living difficulty) sections (preparing for clustering)
"""
DOUT-INDEPENDENT_LIVING_DIFFICULTY:
    0: Yes
    1: No
    Nan: less than 5 years old
"""
ps['DOUT'] = df_person['DOUT'].replace([1, 2], [0, 1])

# Categorizing the DDRS (Ambulatory difficulty) sections (preparing for clustering)
"""
DPHY-AMBULATORY_DIFFICULTY:
    0: Yes
    1: No 
    NaN: less than 5 years old
"""
ps['DPHY'] = df_person['DPHY'].replace([1, 2], [0, 1])

# Categorizing the ENG (Ability to speak english) sections (preparing for clustering)
"""
ENG-ABILITY_TO_SPEAK_ENG
    0: Very well 
    1: Well 
    2: Not well 
    3: Not at all 
    NaN: less than 5 years old
"""
ps['ENG'] = df_person['ENG'].replace([1, 2, 3, 4,], [0, 1, 2, 3])

# Categorizing the MIG (MOBILITY_STATUS) sections (preparing for clustering)
"""
MIG-MOBILITY_STATUS (live here 1 year ago)
    0: Yes, same house 
    1: No, outside US 
    2: No, different house in US 
    NaN: less than 1 year old
"""
ps['MIG'] = df_person['MIG'].replace([1, 2, 3], [0, 1, 2])

In [7]:
# Categorizing mode of transportation to work (preparing for clusterign)
"""
JWTR-MODE_TRANSPORTATION 
   1 - DRIVING
   2 - TRANSIT
   3 - BIKING
   4 - WALKING
"""
ps['JWTR'] = df_person['JWTR'].replace([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], ['DRIVING', 'TRANSIT', 'TRANSIT',
                                                                                   'TRANSIT', 'TRANSIT', 'TRANSIT',
                                                                                   'TRANSIT', 'BIKING', 'BIKING', 'WALKING',
                                                                                   np.nan, np.nan])

# Crating a dummy variable for JWTR (transportation mode to work)
dummy_mode = pd.get_dummies(ps['JWTR'])
dummy_mode.loc[ps['JWTR'].isnull(), :] = np.nan
ps = pd.concat([ps, dummy_mode], axis = 1)

# Categorizing the education section (preparing for clustering)
""" 
SCHL-Education: calculated from education categories using discretion
    0.0 (Less than high school)
    0.0 (High school graduate)
    1.0 (Some college)
    2.0 (Vocational/technical training)
    2.0 (Associates degree)
    4.0 (Bachelor degree)
    6.0 (Graduate/post-graduate degree)
    NaN -> missing or N/A
"""
ps['SCHL'] = df_person['SCHL'].replace([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17,
                                        18, 19, 20, 21, 22, 23, 24], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 
                                                                     0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 
                                                                     1.0, 1.0, 2.0, 4.0, 6.0, 6.0, 6.0])

### Processing Household level data

In [9]:
# Filter the data set for important indices to track
df_household = df_household[['SERIALNO','HINCP','NP','WGTP']]
# Filter for income > 1
df_household = df_household[(df_household["HINCP"] > 1)]

# Escalate income to current year (2018)
# ref http://www.seattle.gov/financedepartment/cpi/documents/US_CPI_History_--_Annual.pdf
df_household['HINCP']  = df_household['HINCP'] * 245.120 / 234.067
                        
# Limit very large households to 8 ppl to correspond with AMI tables
df_household['NP'] = np.where(df_household['NP'] > 8,8,df_household['NP'])

### Personal + Household data

In [12]:
# Summarize personal level data to household level, by calculating means of each features
ps_household = ps.groupby(['SERIALNO'], as_index=False).agg({'DOWNTOWN':'mean', 'NORTHEAST': 'mean',
                                                            'NORTHWEST':'mean', 'SOUTHEAST':'mean', 'WEST':'mean', 'AGEP':'mean',
                                                            'DDRS':'mean', 'ENG':'mean', 'DEAR':'mean', 'DEYE':'mean', 'DOUT':'mean',
                                                            'MIG':'mean', 'HISP':'mean', 'JWMNP':'mean', 'BIKING':'mean', 'DRIVING':'mean', 
                                                            'TRANSIT':'mean', 'WALKING':'mean', 'MALE':'mean', 'FEMALE':'mean', 'SCHL':'mean',
                                                            'WHITE_ALONE':'mean', 'BLACK_ALONE':'mean', 'NATIVE_INDIAN':'mean', 'ASIAN_ALONE':'mean',
                                                            'PACIFIC_ISLANDER':'mean', 'OTHER_RACE':'mean'})

In [13]:
# Merge person level data with household level
df_hh_ps = pd.merge(left=df_household, right=ps_household, how='inner', 
                      left_on='SERIALNO', right_on = 'SERIALNO')
df_hh_ps.head()

,SERIALNO,HINCP,NP,WGTP,DOWNTOWN,NORTHEAST,NORTHWEST,SOUTHEAST,WEST,AGEP,...,WALKING,MALE,FEMALE,SCHL,WHITE_ALONE,BLACK_ALONE,NATIVE_INDIAN,ASIAN_ALONE,PACIFIC_ISLANDER,OTHER_RACE
0,127,17593.321570,2,122,0.0,0.0,0.0,1.0,0.0,56.000000,...,NaN,0.000000,1.000000,1.500000,0.0,0.0,0.0,1.0,0.0,0.0
1,747,246097.057680,4,74,0.0,0.0,0.0,1.0,0.0,21.750000,...,0.0,0.750000,0.250000,2.500000,0.0,0.0,0.0,1.0,0.0,0.0
2,1984,55502.740668,3,69,0.0,1.0,0.0,0.0,0.0,21.666667,...,0.0,0.666667,0.333333,2.000000,1.0,0.0,0.0,0.0,0.0,0.0
3,2319,39794.417838,3,89,0.0,0.0,0.0,0.0,1.0,38.000000,...,0.0,0.666667,0.333333,0.666667,0.0,0.0,0.0,1.0,0.0,0.0
4,2975,78436.892001,3,77,0.0,1.0,0.0,0.0,0.0,38.000000,...,0.0,0.000000,1.000000,2.333333,1.0,0.0,0.0,0.0,0.0,0.0


In [14]:
# Replacing NaNs with the average of each column/attributes for each columns that have Nans 
df_hh_ps['BIKING'] = df_hh_ps['BIKING'].fillna(df_hh_ps["BIKING"].mean())
df_hh_ps['TRANSIT'] = df_hh_ps['TRANSIT'].fillna(df_hh_ps["TRANSIT"].mean())
df_hh_ps['DRIVING'] = df_hh_ps['DRIVING'].fillna(df_hh_ps["DRIVING"].mean())
df_hh_ps['WALKING'] = df_hh_ps['WALKING'].fillna(df_hh_ps["TRANSIT"].mean())
df_hh_ps['ENG'] = df_hh_ps['ENG'].fillna(df_hh_ps["ENG"].mean())

In [15]:
# Checking if there is any NaN (this will be helpful before clustering)
df_hh_ps.isnull().any()

SERIALNO            False
HINCP               False
NP                  False
WGTP                False
DOWNTOWN            False
NORTHEAST           False
NORTHWEST           False
SOUTHEAST           False
WEST                False
AGEP                False
DDRS                False
ENG                 False
DEAR                False
DEYE                False
DOUT                False
MIG                 False
HISP                False
JWMNP               False
BIKING              False
DRIVING             False
TRANSIT             False
WALKING             False
MALE                False
FEMALE              False
SCHL                False
WHITE_ALONE         False
BLACK_ALONE         False
NATIVE_INDIAN       False
ASIAN_ALONE         False
PACIFIC_ISLANDER    False
OTHER_RACE          False
dtype: bool

In [8]:
#df_hh_ps.head(-5)

In [17]:
df_hh_ps.to_csv('pums_processed.csv', index_label=False)